In [ ]:
%pip install optuna
%pip install sparsemax

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sparsemax import Sparsemax
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.datasets import fetch_openml
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
from tqdm import tqdm
import optuna
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sparsemax = Sparsemax(dim=-1)

class TabularDataset(Dataset):
    """Dataset for tabular data with categorical and continuous features"""
    def __init__(self, X_cont, X_cat, y=None, cat_cardinalities=None):
        # Continuous features
        if X_cont is not None and len(X_cont) > 0:
            self.X_cont = torch.tensor(X_cont, dtype=torch.float32)
            self.has_cont = True
        else:
            self.has_cont = False

        # Categorical features
        if X_cat is not None and len(X_cat) > 0:
            self.X_cat = torch.tensor(X_cat, dtype=torch.long)
            self.has_cat = True
        else:
            self.has_cat = False

        # Target
        if y is not None:
            self.y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)
        else:
            self.y = None

        # Store cardinalities for reference
        self.cat_cardinalities = cat_cardinalities

    def __len__(self):
        if self.has_cont:
            return len(self.X_cont)
        else:
            return len(self.X_cat)

    def __getitem__(self, idx):
        data = {}
        if self.has_cont:
            data['cont'] = self.X_cont[idx]
        if self.has_cat:
            data['cat'] = self.X_cat[idx]

        if self.y is not None:
            return data, self.y[idx]
        else:
            return data

class CategoricalEmbeddings(nn.Module):
    """Entity embeddings for categorical features"""
    def __init__(self, cardinalities, d_token):
        super().__init__()
        self.embeddings = nn.ModuleList()
        self.cardinalities = cardinalities

        for cardinality in cardinalities:
            self.embeddings.append(nn.Embedding(cardinality, d_token))

    def forward(self, x):
        # x shape: (batch_size, num_cat_features)
        batch_size = x.shape[0]
        num_features = x.shape[1]

        # Embed each categorical feature
        embedded = []
        for i in range(num_features):
            embedded.append(self.embeddings[i](x[:, i]))

        # Stack along feature dimension
        return torch.stack(embedded, dim=1)  # (batch_size, num_features, d_token)

class LinearEmbedding(nn.Module):
    """Linear embedding for continuous features"""
    def __init__(self, num_features, d_token):
        super().__init__()
        self.embeddings = nn.ModuleList([
            nn.Linear(1, d_token) for _ in range(num_features)
        ])

    def forward(self, x):
        # x shape: (batch_size, num_cont_features)
        batch_size = x.shape[0]
        num_features = x.shape[1]

        # Embed each continuous feature
        embedded = torch.zeros((batch_size, num_features, self.embeddings[0].out_features),
                              device=x.device)

        for i in range(num_features):
            embedded[:, i] = self.embeddings[i](x[:, i].unsqueeze(-1)).squeeze(-1)

        return embedded  # (batch_size, num_features, d_token)

class PiecewiseLinearEmbedding(nn.Module):
    """Piecewise linear embedding for continuous features"""
    def __init__(self, num_features, d_token, num_bins=20):
        super().__init__()
        self.num_features = num_features
        self.d_token = d_token
        self.num_bins = num_bins

        # Create embeddings for each feature
        self.embeddings = nn.ModuleList([
            nn.Linear(num_bins, d_token) for _ in range(num_features)
        ])

        # Create parameters for bin boundaries (learnable)
        self.bin_boundaries = nn.Parameter(torch.randn(num_features, num_bins-1))

    def forward(self, x):
        # x shape: (batch_size, num_features)
        batch_size = x.shape[0]

        # Output will contain embedded tokens for each feature
        embedded = torch.zeros((batch_size, self.num_features, self.d_token), device=x.device)

        for i in range(self.num_features):
            # Get feature values for current feature
            feature_values = x[:, i].unsqueeze(1)  # (batch_size, 1)

            # Get sorted boundaries for this feature
            boundaries = torch.sort(self.bin_boundaries[i]).values  # (num_bins-1)

            # Calculate bin activations using cumulative distribution
            # Start with all in the first bin
            bin_activations = torch.ones((batch_size, self.num_bins), device=x.device)

            # Update bin activations based on feature values and boundaries
            for j in range(self.num_bins-1):
                boundary = boundaries[j]
                # Calculate contribution to bins based on boundary comparison
                condition = feature_values > boundary
                # Move activations to next bin when condition is true
                bin_activations[:, j+1:] = torch.where(
                    condition.expand(-1, self.num_bins-j-1),
                    bin_activations[:, j:self.num_bins-1],
                    bin_activations[:, j+1:]
                )
                bin_activations[:, j] = torch.where(
                    condition.squeeze(1),
                    0.0,
                    bin_activations[:, j]
                )

            # Apply linear transformation to get embeddings
            feature_embedding = self.embeddings[i](bin_activations)  # (batch_size, d_token)
            embedded[:, i] = feature_embedding

        return embedded  # (batch_size, num_features, d_token)

# Custom attention module to capture attention weights
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads

        # Ensure d_model is divisible by num_heads
        assert self.head_dim * num_heads == d_model, "d_model must be divisible by num_heads"

        # Linear projections
        self.q_proj = nn.Linear(d_model, d_model)
        self.k_proj = nn.Linear(d_model, d_model)
        self.v_proj = nn.Linear(d_model, d_model)
        self.out_proj = nn.Linear(d_model, d_model)

        # For storing attention weights
        self.attention_weights = None

    def forward(self, query, key, value, attn_mask=None):
        batch_size = query.shape[0]

        # Linear projections
        q = self.q_proj(query)  # (batch_size, seq_len, d_model)
        k = self.k_proj(key)    # (batch_size, seq_len, d_model)
        v = self.v_proj(value)  # (batch_size, seq_len, d_model)

        # Reshape for multi-head attention
        q = q.view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        k = k.view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)

        # Calculate attention scores
        scores = torch.matmul(q, k.transpose(-2, -1)) / (self.head_dim ** 0.5)

        # Apply mask if provided
        if attn_mask is not None:
            scores = scores.masked_fill(attn_mask == 0, -1e9)

        # Apply softmax to get attention weights
        attention_weights = F.softmax(scores, dim=-1)
        self.attention_weights = attention_weights  # Store for later use

        # Apply attention weights to values
        out = torch.matmul(attention_weights, v)  # (batch_size, num_heads, seq_len, head_dim)

        # Reshape back
        out = out.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)

        # Final linear projection
        out = self.out_proj(out)

        return out

# Custom transformer layer to capture attention weights
class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, nhead)

        # Feed-forward network
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        # Layer norm
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

        # Dropout
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src, src_mask=None):
        # Self-attention
        attn_output = self.self_attn(src, src, src, attn_mask=src_mask)
        src = src + self.dropout1(attn_output)
        src = self.norm1(src)

        # Feed-forward network
        ff_output = self.linear2(self.dropout(F.relu(self.linear1(src))))
        src = src + self.dropout2(ff_output)
        src = self.norm2(src)

        return src

class FTTransformer(nn.Module):
    def __init__(self, cat_cardinalities=None, num_continuous=0, d_token=64, num_heads=8, num_layers=2,
                 d_ffn=128, dropout=0.1, cont_embedding_type='linear', n_bins=20):
        super().__init__()
        self.d_token = d_token
        self.embedding_type = cont_embedding_type

        # Feature counts
        self.num_categorical = len(cat_cardinalities) if cat_cardinalities else 0
        self.num_continuous = num_continuous
        self.total_features = self.num_categorical + self.num_continuous

        # CLS token parameter
        self.cls_token = nn.Parameter(torch.randn(1, 1, d_token))

        # Categorical feature tokenizer
        if self.num_categorical > 0:
            self.cat_tokenizer = CategoricalEmbeddings(cat_cardinalities, d_token)

        # Continuous feature tokenizer
        if self.num_continuous > 0:
            if cont_embedding_type == 'linear':
                self.cont_tokenizer = LinearEmbedding(num_continuous, d_token)
            elif cont_embedding_type == 'piecewise':
                self.cont_tokenizer = PiecewiseLinearEmbedding(num_continuous, d_token, num_bins=n_bins)
            else:
                raise ValueError(f"Unknown continuous embedding type: {cont_embedding_type}")

        # Feature positional embedding
        self.feature_pos_embedding = nn.Parameter(torch.randn(1, self.total_features, d_token))

        # Custom transformer layers
        self.transformer_layers = nn.ModuleList([
            TransformerEncoderLayer(d_model=d_token, nhead=num_heads,
                                   dim_feedforward=d_ffn, dropout=dropout)
            for _ in range(num_layers)
        ])

        # Output layer for classification (sigmoid applied in forward method)
        self.output_layer = nn.Linear(d_token, 1)

    def forward(self, x):
        batch_size = x['cat'].shape[0] if 'cat' in x else x['cont'].shape[0]

        # Tokenize categorical features
        if self.num_categorical > 0 and 'cat' in x:
            cat_tokens = self.cat_tokenizer(x['cat'])  # (batch_size, num_cat, d_token)
        else:
            cat_tokens = None

        # Tokenize continuous features
        if self.num_continuous > 0 and 'cont' in x:
            cont_tokens = self.cont_tokenizer(x['cont'])  # (batch_size, num_cont, d_token)
        else:
            cont_tokens = None

        # Concatenate embeddings from different feature types
        if cat_tokens is not None and cont_tokens is not None:
            tokens = torch.cat([cat_tokens, cont_tokens], dim=1)  # (batch_size, total_features, d_token)
        elif cat_tokens is not None:
            tokens = cat_tokens
        elif cont_tokens is not None:
            tokens = cont_tokens
        else:
            raise ValueError("No features provided")

        # Add positional embedding
        tokens = tokens + self.feature_pos_embedding

        # Add CLS token
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        sequence = torch.cat([cls_tokens, tokens], dim=1)  # (batch_size, num_features+1, d_token)

        # Apply transformer layers
        for layer in self.transformer_layers:
            sequence = layer(sequence)

        # Use CLS token for prediction
        cls_output = sequence[:, 0]

        # Final prediction (logits) - no sigmoid here for BCEWithLogitsLoss
        output = self.output_layer(cls_output)

        return output

    def get_cls_attention(self):
        """Return the attention weights from CLS token to feature tokens (average over all layers)"""
        # Average attention weights across all layers
        cls_attention = []

        for layer in self.transformer_layers:
            # Extract CLS token attention to features
            # layer_weights shape: (batch_size, num_heads, seq_len, seq_len)
            if layer.self_attn.attention_weights is not None:
                # Get attention from CLS (idx 0) to features (idx 1:)
                layer_weights = layer.self_attn.attention_weights
                cls_to_features = layer_weights[:, :, 0, 1:].mean(dim=1)  # Average over heads
                cls_attention.append(cls_to_features)
            else:
                raise ValueError("Attention weights not available. Run forward first.")

        # Average over layers
        avg_attention = torch.stack(cls_attention).mean(dim=0)
        return avg_attention

# Sparse attention variants
class sparseMultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads

        # Ensure d_model is divisible by num_heads
        assert self.head_dim * num_heads == d_model, "d_model must be divisible by num_heads"

        # Linear projections
        self.q_proj = nn.Linear(d_model, d_model)
        self.k_proj = nn.Linear(d_model, d_model)
        self.v_proj = nn.Linear(d_model, d_model)
        self.out_proj = nn.Linear(d_model, d_model)

        # For storing attention weights
        self.attention_weights = None

    def forward(self, query, key, value, attn_mask=None):
        batch_size = query.shape[0]

        # Linear projections
        q = self.q_proj(query)  # (batch_size, seq_len, d_model)
        k = self.k_proj(key)    # (batch_size, seq_len, d_model)
        v = self.v_proj(value)  # (batch_size, seq_len, d_model)

        # Reshape for multi-head attention
        q = q.view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        k = k.view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)

        # Calculate attention scores
        scores = torch.matmul(q, k.transpose(-2, -1)) / (self.head_dim ** 0.5)

        # Apply mask if provided
        if attn_mask is not None:
            scores = scores.masked_fill(attn_mask == 0, -1e9)

        # Apply sparsemax to get attention weights
        attention_weights = sparsemax(scores)
        self.attention_weights = attention_weights  # Store for later use

        # Apply attention weights to values
        out = torch.matmul(attention_weights, v)  # (batch_size, num_heads, seq_len, head_dim)

        # Reshape back
        out = out.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)

        # Final linear projection
        out = self.out_proj(out)

        return out

class sparseTransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout=0.1):
        super().__init__()
        self.self_attn = sparseMultiHeadAttention(d_model, nhead)

        # Feed-forward network
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)

        # Layer norm
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

        # Dropout
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src, src_mask=None):
        # Self-attention
        attn_output = self.self_attn(src, src, src, attn_mask=src_mask)
        src = src + self.dropout1(attn_output)
        src = self.norm1(src)

        # Feed-forward network
        ff_output = self.linear2(self.dropout(F.relu(self.linear1(src))))
        src = src + self.dropout2(ff_output)
        src = self.norm2(src)

        return src

class sparseFTTransformer(nn.Module):
    def __init__(self, cat_cardinalities=None, num_continuous=0, d_token=64, num_heads=8, num_layers=2,
                 d_ffn=128, dropout=0.1, cont_embedding_type='linear', n_bins=20):
        super().__init__()
        self.d_token = d_token
        self.embedding_type = cont_embedding_type

        # Feature counts
        self.num_categorical = len(cat_cardinalities) if cat_cardinalities else 0
        self.num_continuous = num_continuous
        self.total_features = self.num_categorical + self.num_continuous

        # CLS token parameter
        self.cls_token = nn.Parameter(torch.randn(1, 1, d_token))

        # Categorical feature tokenizer
        if self.num_categorical > 0:
            self.cat_tokenizer = CategoricalEmbeddings(cat_cardinalities, d_token)

        # Continuous feature tokenizer
        if self.num_continuous > 0:
            if cont_embedding_type == 'linear':
                self.cont_tokenizer = LinearEmbedding(num_continuous, d_token)
            elif cont_embedding_type == 'piecewise':
                self.cont_tokenizer = PiecewiseLinearEmbedding(num_continuous, d_token, num_bins=n_bins)
            else:
                raise ValueError(f"Unknown continuous embedding type: {cont_embedding_type}")

        # Feature positional embedding
        self.feature_pos_embedding = nn.Parameter(torch.randn(1, self.total_features, d_token))

        # Custom transformer layers
        self.transformer_layers = nn.ModuleList([
            sparseTransformerEncoderLayer(d_model=d_token, nhead=num_heads,
                                   dim_feedforward=d_ffn, dropout=dropout)
            for _ in range(num_layers)
        ])

        # Output layer for classification
        self.output_layer = nn.Linear(d_token, 1)

    def forward(self, x):
        batch_size = x['cat'].shape[0] if 'cat' in x else x['cont'].shape[0]

        # Tokenize categorical features
        if self.num_categorical > 0 and 'cat' in x:
            cat_tokens = self.cat_tokenizer(x['cat'])  # (batch_size, num_cat, d_token)
        else:
            cat_tokens = None

        # Tokenize continuous features
        if self.num_continuous > 0 and 'cont' in x:
            cont_tokens = self.cont_tokenizer(x['cont'])  # (batch_size, num_cont, d_token)
        else:
            cont_tokens = None

        # Concatenate embeddings from different feature types
        if cat_tokens is not None and cont_tokens is not None:
            tokens = torch.cat([cat_tokens, cont_tokens], dim=1)  # (batch_size, total_features, d_token)
        elif cat_tokens is not None:
            tokens = cat_tokens
        elif cont_tokens is not None:
            tokens = cont_tokens
        else:
            raise ValueError("No features provided")

        # Add positional embedding
        tokens = tokens + self.feature_pos_embedding

        # Add CLS token
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)
        sequence = torch.cat([cls_tokens, tokens], dim=1)  # (batch_size, num_features+1, d_token)

        # Apply transformer layers
        for layer in self.transformer_layers:
            sequence = layer(sequence)

        # Use CLS token for prediction
        cls_output = sequence[:, 0]

        # Final prediction (logits)
        output = self.output_layer(cls_output)

        return output

    def get_cls_attention(self):
        """Return the attention weights from CLS token to feature tokens (average over all layers)"""
        # Average attention weights across all layers
        cls_attention = []

        for layer in self.transformer_layers:
            # Extract CLS token attention to features
            # layer_weights shape: (batch_size, num_heads, seq_len, seq_len)
            if layer.self_attn.attention_weights is not None:
                # Get attention from CLS (idx 0) to features (idx 1:)
                layer_weights = layer.self_attn.attention_weights
                cls_to_features = layer_weights[:, :, 0, 1:].mean(dim=1)  # Average over heads
                cls_attention.append(cls_to_features)
            else:
                raise ValueError("Attention weights not available. Run forward first.")

        # Average over layers
        avg_attention = torch.stack(cls_attention).mean(dim=0)
        return avg_attention

def calculate_pfi(model, X_cont, X_cat, y, cat_cardinalities, num_permutations=5):
    """Calculate Permutation Feature Importance (PFI) for classification with mixed features"""
    # Convert to PyTorch tensors
    data = {}
    if X_cont is not None and len(X_cont) > 0:
        data['cont'] = torch.tensor(X_cont, dtype=torch.float32).to(device)
        has_cont = True
    else:
        has_cont = False

    if X_cat is not None and len(X_cat) > 0:
        data['cat'] = torch.tensor(X_cat, dtype=torch.long).to(device)
        has_cat = True
    else:
        has_cat = False

    y_tensor = torch.tensor(y, dtype=torch.float32).reshape(-1, 1).to(device)

    # Get baseline performance
    model.eval()
    with torch.no_grad():
        baseline_preds = torch.sigmoid(model(data))
        baseline_loss = F.binary_cross_entropy(baseline_preds, y_tensor).item()
        baseline_preds_binary = (baseline_preds > 0.5).float()
        baseline_accuracy = (baseline_preds_binary == y_tensor).float().mean().item()

    # Calculate importance for each feature
    importances = []

    # First categorical features
    if has_cat:
        for feat_idx in range(X_cat.shape[1]):
            accuracies = []

            for _ in range(num_permutations):
                # Create a permuted copy of the data
                data_permuted = {key: val.clone() if torch.is_tensor(val) else val for key, val in data.items()}

                # Permute the categorical feature
                perm_idx = torch.randperm(X_cat.shape[0])
                data_permuted['cat'][:, feat_idx] = data_permuted['cat'][perm_idx, feat_idx]

                # Calculate loss with permuted feature
                with torch.no_grad():
                    perm_preds = torch.sigmoid(model(data_permuted))
                    perm_preds_binary = (perm_preds > 0.5).float()
                    perm_accuracy = (perm_preds_binary == y_tensor).float().mean().item()

                # Feature importance is the decrease in accuracy
                accuracies.append(baseline_accuracy - perm_accuracy)

            # Average over permutations (higher = more important)
            importances.append(np.mean(accuracies))

    # Then continuous features
    if has_cont:
        for feat_idx in range(X_cont.shape[1]):
            accuracies = []

            for _ in range(num_permutations):
                # Create a permuted copy of the data
                data_permuted = {key: val.clone() if torch.is_tensor(val) else val for key, val in data.items()}

                # Permute the continuous feature
                perm_idx = torch.randperm(X_cont.shape[0])
                data_permuted['cont'][:, feat_idx] = data_permuted['cont'][perm_idx, feat_idx]

                # Calculate loss with permuted feature
                with torch.no_grad():
                    perm_preds = torch.sigmoid(model(data_permuted))
                    perm_preds_binary = (perm_preds > 0.5).float()
                    perm_accuracy = (perm_preds_binary == y_tensor).float().mean().item()

                # Feature importance is the decrease in accuracy
                accuracies.append(baseline_accuracy - perm_accuracy)

            # Average over permutations (higher = more important)
            importances.append(np.mean(accuracies))

    return np.array(importances)

def train_model(model, train_loader, val_loader, criterion, optimizer, device, epochs=100, early_stopping=16):
    """Train the model with early stopping"""
    model.to(device)
    best_val_loss = float('inf')
    early_stop_counter = 0
    best_state = None

    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0

        for X_batch, y_batch in train_loader:
            # Move data to device
            for key in X_batch:
                X_batch[key] = X_batch[key].to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            # Calculate accuracy
            predictions = (torch.sigmoid(outputs) > 0.5).float()
            train_correct += (predictions == y_batch).sum().item()
            train_total += y_batch.size(0)

        # Validation
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        val_preds = []
        val_targets = []

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                # Move data to device
                for key in X_batch:
                    X_batch[key] = X_batch[key].to(device)
                y_batch = y_batch.to(device)

                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()

                # Calculate accuracy
                predictions = (torch.sigmoid(outputs) > 0.5).float()
                val_correct += (predictions == y_batch).sum().item()
                val_total += y_batch.size(0)

                # Store predictions and targets for metrics
                val_preds.append(torch.sigmoid(outputs).cpu())
                val_targets.append(y_batch.cpu())

        train_loss /= len(train_loader)
        train_accuracy = train_correct / train_total
        val_loss /= len(val_loader)
        val_accuracy = val_correct / val_total

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stop_counter = 0
            # Save best model state dict
            best_state = model.state_dict()
        else:
            early_stop_counter += 1
            if early_stop_counter >= early_stopping:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # Load best model
    if best_state is not None:
        model.load_state_dict(best_state)
    return model

def evaluate_model(model, X_cont, X_cat, y, device):
    """Evaluate model performance for classification with mixed features"""
    model.to(device)
    model.eval()

    # Prepare data
    data = {}
    if X_cont is not None and len(X_cont) > 0:
        data['cont'] = torch.tensor(X_cont, dtype=torch.float32).to(device)

    if X_cat is not None and len(X_cat) > 0:
        data['cat'] = torch.tensor(X_cat, dtype=torch.long).to(device)

    y_tensor = torch.tensor(y, dtype=torch.float32).reshape(-1, 1).to(device)

    with torch.no_grad():
        logits = model(data)
        y_pred_proba = torch.sigmoid(logits).cpu().numpy()
        y_pred = (y_pred_proba > 0.5).astype(int)

    y = y.reshape(-1, 1)

    # Calculate metrics
    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    auc = roc_auc_score(y, y_pred_proba)

    # Confusion matrix
    cm = confusion_matrix(y, y_pred)

    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1 Score: {f1:.4f}")
    print(f"Test AUC: {auc:.4f}")
    print(f"Confusion Matrix:\n{cm}")

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc,
        'confusion_matrix': cm.tolist()
    }

def analyze_pfi_attention_correlation(model, X_cont, X_cat, y, cat_feature_names, cont_feature_names, cat_cardinalities, device):
    """Analyze correlation between PFI and attention scores for classification with mixed features"""
    model.to(device)
    model.eval()

    # Prepare data for forward pass
    data = {}
    if X_cont is not None and len(X_cont) > 0:
        data['cont'] = torch.tensor(X_cont, dtype=torch.float32).to(device)

    if X_cat is not None and len(X_cat) > 0:
        data['cat'] = torch.tensor(X_cat, dtype=torch.long).to(device)

    # Get attention scores
    with torch.no_grad():
        _ = model(data)  # Forward pass to compute attention
        attention_scores = model.get_cls_attention().cpu().numpy()

    # Average attention scores across samples
    avg_attention = attention_scores.mean(axis=0)

    # Calculate PFI
    pfi_scores = calculate_pfi(model, X_cont, X_cat, y, cat_cardinalities)

    # Combine feature names in the same order as the attention scores
    # (categorical first, then continuous)
    feature_names = []
    if cat_feature_names:
        feature_names.extend(cat_feature_names)
    if cont_feature_names:
        feature_names.extend(cont_feature_names)

    # Calculate Spearman rank correlation
    correlation, p_value = spearmanr(pfi_scores, avg_attention)

    print(f"Spearman Rank Correlation: {correlation:.4f} (p-value: {p_value:.4f})")

    # Create a visualization
    fig, ax = plt.subplots(figsize=(12, 8))

    # Create a scatter plot with different colors for categorical and continuous features
    scatter_cat = None
    scatter_cont = None

    if cat_feature_names and cont_feature_names:
        n_cat = len(cat_feature_names)
        scatter_cat = ax.scatter(pfi_scores[:n_cat], avg_attention[:n_cat], alpha=0.7,
                                  label='Categorical Features', marker='o', color='blue')
        scatter_cont = ax.scatter(pfi_scores[n_cat:], avg_attention[n_cat:], alpha=0.7,
                                   label='Continuous Features', marker='x', color='red')
    else:
        ax.scatter(pfi_scores, avg_attention, alpha=0.7)

    # Add feature labels
    for i, name in enumerate(feature_names):
        ax.annotate(name, (pfi_scores[i], avg_attention[i]),
                   textcoords="offset points", xytext=(0,10), ha='center', fontsize=8)

    # Add best fit line
    z = np.polyfit(pfi_scores, avg_attention, 1)
    p = np.poly1d(z)
    ax.plot(np.sort(pfi_scores), p(np.sort(pfi_scores)), "r--", alpha=0.7)

    # Add correlation information
    ax.text(0.05, 0.95, f"Spearman ρ: {correlation:.4f}\np-value: {p_value:.4f}",
            transform=ax.transAxes, verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    ax.set_xlabel('Permutation Feature Importance')
    ax.set_ylabel('CLS Token Attention Score')
    ax.set_title('PFI vs CLS Token Attention Correlation')

    if scatter_cat and scatter_cont:
        ax.legend()

    plt.tight_layout()
    plt.savefig('pfi_attention_correlation_adult.png')
    plt.close()

    # Return results
    results = {
        'correlation': correlation,
        'p_value': p_value,
        'pfi_scores': pfi_scores.tolist(),
        'attention_scores': avg_attention.tolist(),
        'feature_names': feature_names
    }

    return results

def load_adult_dataset():
    """Load and preprocess Adult dataset using fetch_openml"""
    print("Loading Adult dataset...")

    # Load the Adult dataset
    adult = fetch_openml(name='adult', version=2, as_frame=True)

    # Get the data and target
    df = adult.data
    y = adult.target

    # Convert target to binary
    y = (y == '>50K').astype(int).values

    # Identify categorical and numerical features
    # Adult dataset has the following features:
    categorical_features = [
        'workclass', 'education', 'marital-status', 'occupation',
        'relationship', 'race', 'sex', 'native-country'
    ]

    numerical_features = [
        'age', 'capital-gain', 'capital-loss', 'hours-per-week', 'fnlwgt', 'education-num'
    ]

    # Handle missing values if any
    for col in categorical_features:
        if df[col].isna().any():
            df[col] = df[col].fillna(df[col].mode()[0])

    for col in numerical_features:
        if df[col].isna().any():
            df[col] = df[col].fillna(df[col].median())

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    # Process categorical features - Label Encoding
    label_encoders = {}
    cat_cardinalities = []

    X_cat_train = np.zeros((len(X_train), len(categorical_features)), dtype=np.int64)
    X_cat_val = np.zeros((len(X_val), len(categorical_features)), dtype=np.int64)
    X_cat_test = np.zeros((len(X_test), len(categorical_features)), dtype=np.int64)

    for i, col in enumerate(categorical_features):
        le = LabelEncoder()
        X_cat_train[:, i] = le.fit_transform(X_train[col])
        X_cat_val[:, i] = le.transform(X_val[col])
        X_cat_test[:, i] = le.transform(X_test[col])

        label_encoders[col] = le
        cat_cardinalities.append(len(le.classes_))

    # Process numerical features - Standardization
    scaler = StandardScaler()
    X_cont_train = scaler.fit_transform(X_train[numerical_features])
    X_cont_val = scaler.transform(X_val[numerical_features])
    X_cont_test = scaler.transform(X_test[numerical_features])

    print(f"Dataset shapes:")
    print(f"  X_cat_train: {X_cat_train.shape}, X_cont_train: {X_cont_train.shape}")
    print(f"  X_cat_val: {X_cat_val.shape}, X_cont_val: {X_cont_val.shape}")
    print(f"  X_cat_test: {X_cat_test.shape}, X_cont_test: {X_cont_test.shape}")
    print(f"Categorical feature cardinalities: {cat_cardinalities}")

    return (X_cont_train, X_cat_train, X_cont_val, X_cat_val, X_cont_test, X_cat_test,
            y_train, y_val, y_test, categorical_features, numerical_features, cat_cardinalities)

def tune_hyperparameters(X_cont_train, X_cat_train, y_train, X_cont_val, X_cat_val, y_val,
                        cat_cardinalities, cont_embedding_type='linear', n_trials=20, sparse=False):
    """Tune hyperparameters using Optuna for mixed feature types"""

    # Create datasets
    train_dataset = TabularDataset(X_cont_train, X_cat_train, y_train, cat_cardinalities)
    val_dataset = TabularDataset(X_cont_val, X_cat_val, y_val, cat_cardinalities)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=128)

    num_categorical = len(cat_cardinalities) if cat_cardinalities else 0
    num_continuous = X_cont_train.shape[1] if X_cont_train is not None else 0

    def objective(trial):
        # Define hyperparameters to tune
        d_token = trial.suggest_int('d_token', 32, 128)
        num_heads = trial.suggest_int('num_heads', 2, 8)
        num_layers = trial.suggest_int('num_layers', 1, 3)
        d_ffn = trial.suggest_int('d_ffn', 64, 256)
        lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
        dropout = trial.suggest_float('dropout', 0.0, 0.5)
        n_bins = trial.suggest_int('n_bins', 10, 100)

        # Ensure d_token is divisible by num_heads
        d_token = (d_token // num_heads) * num_heads

        # Create model with trial hyperparameters
        if not sparse:
            model = FTTransformer(
                cat_cardinalities=cat_cardinalities,
                num_continuous=num_continuous,
                d_token=d_token,
                num_heads=num_heads,
                num_layers=num_layers,
                d_ffn=d_ffn,
                dropout=dropout,
                cont_embedding_type=cont_embedding_type,
                n_bins=n_bins
            )
        else:
            model = sparseFTTransformer(
                cat_cardinalities=cat_cardinalities,
                num_continuous=num_continuous,
                d_token=d_token,
                num_heads=num_heads,
                num_layers=num_layers,
                d_ffn=d_ffn,
                dropout=dropout,
                cont_embedding_type=cont_embedding_type,
                n_bins=n_bins
            )

        # Define criterion and optimizer for binary classification
        criterion = nn.BCEWithLogitsLoss()
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-5)

        # Train for a few epochs
        model.to(device)
        best_val_loss = float('inf')

        patience = 5
        patience_counter = 0
        num_epochs = 20

        # Short training loop for hyperparameter search
        for epoch in range(num_epochs):
            # Training
            model.train()
            for X_batch, y_batch in train_loader:
                # Move data to device
                for key in X_batch:
                    X_batch[key] = X_batch[key].to(device)
                y_batch = y_batch.to(device)

                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()

            # Validation
            model.eval()
            val_loss = 0
            val_correct = 0
            val_total = 0

            with torch.no_grad():
                for X_batch, y_batch in val_loader:
                    # Move data to device
                    for key in X_batch:
                        X_batch[key] = X_batch[key].to(device)
                    y_batch = y_batch.to(device)

                    outputs = model(X_batch)
                    loss = criterion(outputs, y_batch)
                    val_loss += loss.item()

                    # Calculate accuracy
                    predictions = (torch.sigmoid(outputs) > 0.5).float()
                    val_correct += (predictions == y_batch).sum().item()
                    val_total += y_batch.size(0)

            val_loss /= len(val_loader)
            val_accuracy = val_correct / val_total

            # Update best validation loss
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter > patience:
                    break


        return best_val_loss

    # Create Optuna study
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=n_trials, timeout=1800)

    # Print best parameters
    print("Best trial:")
    trial = study.best_trial
    print(f"  Value (validation loss): {trial.value:.4f}")
    print("  Params:")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

    # Return best parameters
    return trial.params

def train_with_best_params(X_cont_train, X_cat_train, y_train, X_cont_val, X_cat_val, y_val,
                          X_cont_test, X_cat_test, y_test, cat_cardinalities, best_params,
                          cont_embedding_type='linear', sparse=False):
    """Train a model with the best hyperparameters for mixed feature types"""
    # Create datasets
    train_dataset = TabularDataset(X_cont_train, X_cat_train, y_train, cat_cardinalities)
    val_dataset = TabularDataset(X_cont_val, X_cat_val, y_val, cat_cardinalities)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=1024)

    # Ensure d_token is divisible by num_heads
    d_token = (best_params['d_token'] // best_params['num_heads']) * best_params['num_heads']

    num_continuous = X_cont_train.shape[1] if X_cont_train is not None else 0

    # Create model with best hyperparameters
    if not sparse:
        model = FTTransformer(
            cat_cardinalities=cat_cardinalities,
            num_continuous=num_continuous,
            d_token=d_token,
            num_heads=best_params['num_heads'],
            num_layers=best_params['num_layers'],
            d_ffn=best_params['d_ffn'],
            dropout=best_params['dropout'],
            cont_embedding_type=cont_embedding_type,
            n_bins=best_params['n_bins']
        )
    else:
        model = sparseFTTransformer(
            cat_cardinalities=cat_cardinalities,
            num_continuous=num_continuous,
            d_token=d_token,
            num_heads=best_params['num_heads'],
            num_layers=best_params['num_layers'],
            d_ffn=best_params['d_ffn'],
            dropout=best_params['dropout'],
            cont_embedding_type=cont_embedding_type,
            n_bins=best_params['n_bins']
        )

    # Define criterion for binary classification
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=best_params['lr'], weight_decay=1e-5)

    # Train the model with early stopping
    model = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        epochs=100
    )

    # Evaluate on test set
    results = evaluate_model(model, X_cont_test, X_cat_test, y_test, device)

    return model, results

def visualize_all_models(results, cat_feature_names, cont_feature_names):
    """Create a comprehensive visualization comparing all models with mixed feature types"""
    all_feature_names = []
    if cat_feature_names:
        all_feature_names.extend(cat_feature_names)
    if cont_feature_names:
        all_feature_names.extend(cont_feature_names)

    # Create a figure with 2x2 subplots
    fig, axs = plt.subplots(2, 2, figsize=(20, 16))

    # Plot performance metrics
    models = list(results.keys())
    acc_values = [results[model]['accuracy'] for model in models]
    f1_values = [results[model]['f1'] for model in models]

    # Accuracy comparison
    axs[0, 0].bar(models, acc_values)
    axs[0, 0].set_title('Accuracy Comparison (Adult)')
    axs[0, 0].set_ylabel('Accuracy')
    axs[0, 0].tick_params(axis='x', rotation=45)

    # F1 comparison
    axs[0, 1].bar(models, f1_values)
    axs[0, 1].set_title('F1 Score Comparison (Adult)')
    axs[0, 1].set_ylabel('F1 Score')
    axs[0, 1].tick_params(axis='x', rotation=45)

    # Correlation comparison
    correlations = [results[model]['correlation_analysis']['correlation'] for model in models]
    p_values = [results[model]['correlation_analysis']['p_value'] for model in models]

    axs[1, 0].bar(models, correlations)
    axs[1, 0].set_title('PFI-Attention Correlation Comparison (Adult)')
    axs[1, 0].set_ylabel('Spearman Correlation')
    axs[1, 0].tick_params(axis='x', rotation=45)

    # Feature importance comparison across models (top 3 features)
    axs[1, 1].axis('off')  # Turn off the axis for the text summary

    summary_text = "Feature Importance Summary:\n\n"

    for model in models:
        pfi_scores = np.array(results[model]['correlation_analysis']['pfi_scores'])
        attn_scores = np.array(results[model]['correlation_analysis']['attention_scores'])
        feature_names = results[model]['correlation_analysis']['feature_names']

        # Get top 3 features by PFI
        pfi_top_indices = np.argsort(-pfi_scores)[:3]
        pfi_top_features = [feature_names[i] for i in pfi_top_indices]

        # Get top 3 features by attention
        attn_top_indices = np.argsort(-attn_scores)[:3]
        attn_top_features = [feature_names[i] for i in attn_top_indices]

        summary_text += f"{model}:\n"
        summary_text += f"  Top PFI features: {', '.join(pfi_top_features)}\n"
        summary_text += f"  Top attention features: {', '.join(attn_top_features)}\n\n"

    axs[1, 1].text(0.05, 0.95, summary_text, transform=axs[1, 1].transAxes,
                 verticalalignment='top', fontsize=12)

    plt.tight_layout()
    plt.savefig('model_comparison_adult.png')
    plt.close()

    # Create additional visualization for feature importance comparison
    # For top 20 features only to avoid cluttering
    num_top_features = min(20, len(all_feature_names))
    fig, axs = plt.subplots(len(models), 1, figsize=(14, 5 * len(models)))

    if len(models) == 1:
        axs = [axs]  # Convert to list if there's only one model

    for i, model in enumerate(models):
        pfi_scores = np.array(results[model]['correlation_analysis']['pfi_scores'])
        attn_scores = np.array(results[model]['correlation_analysis']['attention_scores'])
        feature_names = results[model]['correlation_analysis']['feature_names']

        # Sort features by PFI for visualization (top 20)
        sorted_indices = np.argsort(-pfi_scores)[:num_top_features]
        sorted_features = [feature_names[j] for j in sorted_indices]
        sorted_pfi = [pfi_scores[j] for j in sorted_indices]
        sorted_attn = [attn_scores[j] for j in sorted_indices]

        x = np.arange(len(sorted_features))
        width = 0.35

        # Color-code categorical vs. continuous features
        colors_pfi = []
        colors_attn = []
        n_cat = len(cat_feature_names) if cat_feature_names else 0

        for feat in sorted_features:
            if feat in cat_feature_names:
                colors_pfi.append('blue')
                colors_attn.append('lightblue')
            else:
                colors_pfi.append('red')
                colors_attn.append('lightcoral')

        axs[i].bar(x - width/2, sorted_pfi, width, label='PFI', color=colors_pfi)
        axs[i].bar(x + width/2, sorted_attn, width, label='Attention', color=colors_attn)

        axs[i].set_title(f'Feature Importance (Adult): {model}')
        axs[i].set_ylabel('Importance Score')
        axs[i].set_xticks(x)
        axs[i].set_xticklabels(sorted_features, rotation=45, ha='right')

        # Add legend with categorical/continuous indicators
        from matplotlib.patches import Patch
        legend_elements = [
            Patch(facecolor='blue', label='Cat. PFI'),
            Patch(facecolor='lightblue', label='Cat. Attention'),
            Patch(facecolor='red', label='Cont. PFI'),
            Patch(facecolor='lightcoral', label='Cont. Attention')
        ]
        axs[i].legend(handles=legend_elements)

    plt.tight_layout()
    plt.savefig('feature_importance_comparison_adult.png')
    plt.close()

    print("\nVisualizations saved as 'model_comparison_adult.png' and 'feature_importance_comparison_adult.png'")

def save_model(model, filename):
    torch.save(model.state_dict(), filename)
    print(f"Model saved as {filename}")

def save_results(results, filename):
    # Convert numpy arrays to lists for json serialization
    for model in results:
        if 'correlation_analysis' in results[model]:
            if isinstance(results[model]['correlation_analysis']['pfi_scores'], np.ndarray):
                results[model]['correlation_analysis']['pfi_scores'] = results[model]['correlation_analysis']['pfi_scores'].tolist()
            if isinstance(results[model]['correlation_analysis']['attention_scores'], np.ndarray):
                results[model]['correlation_analysis']['attention_scores'] = results[model]['correlation_analysis']['attention_scores'].tolist()

    with open(filename, 'w') as f:
        json.dump(results, f, indent=2)
    print(f"Results saved as {filename}")

def main_with_tuning():
    """Main function to run the Adult dataset experiments with categorical and continuous features"""
    # Set random seed for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)

    # Set device
    print(f"Using device: {device}")

    # Load Adult dataset with proper feature separation
    (X_cont_train, X_cat_train, X_cont_val, X_cat_val, X_cont_test, X_cat_test,
     y_train, y_val, y_test, categorical_features, numerical_features,
     cat_cardinalities) = load_adult_dataset()

    models = {}
    results = {}

    # Tune hyperparameters for Linear Embedding
    print("\n=== Tuning Hyperparameters for FT Transformer with Linear Embedding ===")
    linear_best_params = tune_hyperparameters(
        X_cont_train=X_cont_train,
        X_cat_train=X_cat_train,
        y_train=y_train,
        X_cont_val=X_cont_val,
        X_cat_val=X_cat_val,
        y_val=y_val,
        cat_cardinalities=cat_cardinalities,
        cont_embedding_type='linear',
        n_trials=20
    )

    # Train with best parameters for Linear Embedding
    print("\n=== Training FT Transformer with Linear Embedding (Tuned) ===")
    ft_linear_tuned, linear_results = train_with_best_params(
        X_cont_train=X_cont_train,
        X_cat_train=X_cat_train,
        y_train=y_train,
        X_cont_val=X_cont_val,
        X_cat_val=X_cat_val,
        y_val=y_val,
        X_cont_test=X_cont_test,
        X_cat_test=X_cat_test,
        y_test=y_test,
        cat_cardinalities=cat_cardinalities,
        best_params=linear_best_params,
        cont_embedding_type='linear'
    )

    save_model(ft_linear_tuned, 'ft_linear_tuned_adult.pth')

    models['ft_linear_tuned'] = ft_linear_tuned
    results['ft_linear_tuned'] = linear_results

    # Analyze PFI and attention correlation for tuned linear model
    print("\n=== Analyzing PFI vs Attention Correlation for Linear Embedding (Tuned) ===")
    linear_tuned_correlation = analyze_pfi_attention_correlation(
        model=ft_linear_tuned,
        X_cont=X_cont_val,
        X_cat=X_cat_val,
        y=y_val,
        cat_feature_names=categorical_features,
        cont_feature_names=numerical_features,
        cat_cardinalities=cat_cardinalities,
        device=device
    )
    results['ft_linear_tuned']['correlation_analysis'] = linear_tuned_correlation

    # Tune hyperparameters for Piecewise Linear Embedding
    print("\n=== Tuning Hyperparameters for FT Transformer with Piecewise Linear Embedding ===")
    piecewise_best_params = tune_hyperparameters(
        X_cont_train=X_cont_train,
        X_cat_train=X_cat_train,
        y_train=y_train,
        X_cont_val=X_cont_val,
        X_cat_val=X_cat_val,
        y_val=y_val,
        cat_cardinalities=cat_cardinalities,
        cont_embedding_type='piecewise',
        n_trials=20
    )

    # Train with best parameters for Piecewise Linear Embedding
    print("\n=== Training FT Transformer with Piecewise Linear Embedding (Tuned) ===")
    ft_piecewise_tuned, piecewise_results = train_with_best_params(
        X_cont_train=X_cont_train,
        X_cat_train=X_cat_train,
        y_train=y_train,
        X_cont_val=X_cont_val,
        X_cat_val=X_cat_val,
        y_val=y_val,
        X_cont_test=X_cont_test,
        X_cat_test=X_cat_test,
        y_test=y_test,
        cat_cardinalities=cat_cardinalities,
        best_params=piecewise_best_params,
        cont_embedding_type='piecewise'
    )

    save_model(ft_piecewise_tuned, 'ft_piecewise_tuned_adult.pth')

    models['ft_piecewise_tuned'] = ft_piecewise_tuned
    results['ft_piecewise_tuned'] = piecewise_results

    # Analyze PFI and attention correlation for tuned piecewise model
    print("\n=== Analyzing PFI vs Attention Correlation for Piecewise Embedding (Tuned) ===")
    piecewise_tuned_correlation = analyze_pfi_attention_correlation(
        model=ft_piecewise_tuned,
        X_cont=X_cont_val,
        X_cat=X_cat_val,
        y=y_val,
        cat_feature_names=categorical_features,
        cont_feature_names=numerical_features,
        cat_cardinalities=cat_cardinalities,
        device=device
    )
    results['ft_piecewise_tuned']['correlation_analysis'] = piecewise_tuned_correlation

    # Tune hyperparameters for Sparse Linear Embedding
    print("\n=== Tuning Hyperparameters for sparse FT Transformer with Linear Embedding ===")
    sparse_linear_best_params = tune_hyperparameters(
        X_cont_train=X_cont_train,
        X_cat_train=X_cat_train,
        y_train=y_train,
        X_cont_val=X_cont_val,
        X_cat_val=X_cat_val,
        y_val=y_val,
        cat_cardinalities=cat_cardinalities,
        cont_embedding_type='linear',
        n_trials=20,
        sparse=True
    )

    # Train with best parameters for Sparse Linear Embedding
    print("\n=== Training sparse FT Transformer with Linear Embedding (Tuned) ===")
    sparse_ft_linear_tuned, sparse_linear_results = train_with_best_params(
        X_cont_train=X_cont_train,
        X_cat_train=X_cat_train,
        y_train=y_train,
        X_cont_val=X_cont_val,
        X_cat_val=X_cat_val,
        y_val=y_val,
        X_cont_test=X_cont_test,
        X_cat_test=X_cat_test,
        y_test=y_test,
        cat_cardinalities=cat_cardinalities,
        best_params=sparse_linear_best_params,
        cont_embedding_type='linear',
        sparse=True
    )

    save_model(sparse_ft_linear_tuned, 'sparse_ft_linear_tuned_adult.pth')

    models['sparse_ft_linear_tuned'] = sparse_ft_linear_tuned
    results['sparse_ft_linear_tuned'] = sparse_linear_results

    # Analyze PFI and attention correlation for tuned sparse linear model
    print("\n=== Analyzing PFI vs Attention Correlation for Sparse Linear Embedding (Tuned) ===")
    sparse_linear_tuned_correlation = analyze_pfi_attention_correlation(
        model=sparse_ft_linear_tuned,
        X_cont=X_cont_val,
        X_cat=X_cat_val,
        y=y_val,
        cat_feature_names=categorical_features,
        cont_feature_names=numerical_features,
        cat_cardinalities=cat_cardinalities,
        device=device
    )
    results['sparse_ft_linear_tuned']['correlation_analysis'] = sparse_linear_tuned_correlation

    # Tune hyperparameters for Sparse Piecewise Embedding
    print("\n=== Tuning Hyperparameters for sparse FT Transformer with Piecewise Linear Embedding ===")
    sparse_piecewise_best_params = tune_hyperparameters(
        X_cont_train=X_cont_train,
        X_cat_train=X_cat_train,
        y_train=y_train,
        X_cont_val=X_cont_val,
        X_cat_val=X_cat_val,
        y_val=y_val,
        cat_cardinalities=cat_cardinalities,
        cont_embedding_type='piecewise',
        n_trials=20,
        sparse=True
    )

    # Train with best parameters for Sparse Piecewise Embedding
    print("\n=== Training sparse FT Transformer with Piecewise Linear Embedding (Tuned) ===")
    sparse_ft_piecewise_tuned, sparse_piecewise_results = train_with_best_params(
        X_cont_train=X_cont_train,
        X_cat_train=X_cat_train,
        y_train=y_train,
        X_cont_val=X_cont_val,
        X_cat_val=X_cat_val,
        y_val=y_val,
        X_cont_test=X_cont_test,
        X_cat_test=X_cat_test,
        y_test=y_test,
        cat_cardinalities=cat_cardinalities,
        best_params=sparse_piecewise_best_params,
        cont_embedding_type='piecewise',
        sparse=True
    )

    save_model(sparse_ft_piecewise_tuned, 'sparse_ft_piecewise_tuned_adult.pth')

    models['sparse_ft_piecewise_tuned'] = sparse_ft_piecewise_tuned
    results['sparse_ft_piecewise_tuned'] = sparse_piecewise_results

    # Analyze PFI and attention correlation for tuned sparse piecewise model
    print("\n=== Analyzing PFI vs Attention Correlation for Sparse Piecewise Embedding (Tuned) ===")
    sparse_piecewise_tuned_correlation = analyze_pfi_attention_correlation(
        model=sparse_ft_piecewise_tuned,
        X_cont=X_cont_val,
        X_cat=X_cat_val,
        y=y_val,
        cat_feature_names=categorical_features,
        cont_feature_names=numerical_features,
        cat_cardinalities=cat_cardinalities,
        device=device
    )
    results['sparse_ft_piecewise_tuned']['correlation_analysis'] = sparse_piecewise_tuned_correlation

    # Compare the results
    print("\n=== Comparison of Tuned Models ===")
    print(f"FT Transformer (Linear Tuned): Accuracy={results['ft_linear_tuned']['accuracy']:.4f}, F1={results['ft_linear_tuned']['f1']:.4f}")
    print(f"FT Transformer (Piecewise Tuned): Accuracy={results['ft_piecewise_tuned']['accuracy']:.4f}, F1={results['ft_piecewise_tuned']['f1']:.4f}")
    print(f"Sparse FT Transformer (Linear Tuned): Accuracy={results['sparse_ft_linear_tuned']['accuracy']:.4f}, F1={results['sparse_ft_linear_tuned']['f1']:.4f}")
    print(f"Sparse FT Transformer (Piecewise Tuned): Accuracy={results['sparse_ft_piecewise_tuned']['accuracy']:.4f}, F1={results['sparse_ft_piecewise_tuned']['f1']:.4f}")

    print("\n=== Comparison of PFI-Attention Correlations (Tuned Models) ===")
    print(f"FT Transformer (Linear Tuned): ρ={linear_tuned_correlation['correlation']:.4f}, p-value={linear_tuned_correlation['p_value']:.4f}")
    print(f"FT Transformer (Piecewise Tuned): ρ={piecewise_tuned_correlation['correlation']:.4f}, p-value={piecewise_tuned_correlation['p_value']:.4f}")
    print(f"Sparse FT Transformer (Linear Tuned): ρ={sparse_linear_tuned_correlation['correlation']:.4f}, p-value={sparse_linear_tuned_correlation['p_value']:.4f}")
    print(f"Sparse FT Transformer (Piecewise Tuned): ρ={sparse_piecewise_tuned_correlation['correlation']:.4f}, p-value={sparse_piecewise_tuned_correlation['p_value']:.4f}")

    # Create visualization comparing all models
    visualize_all_models(
        results=results,
        cat_feature_names=categorical_features,
        cont_feature_names=numerical_features
    )

    save_results(results, 'results_adult.json')

    return models, results

if __name__ == "__main__":
    main_with_tuning()

Using device: cuda
Loading Adult dataset...


<ipython-input-2-0437500be3f4>:884: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].fillna(df[col].mode()[0])
[I 2025-03-04 15:57:10,160] A new study created in memory with name: no-name-a77e7a77-88a8-42ba-b79c-d88f0b7629bc


Dataset shapes:
  X_cat_train: (31258, 8), X_cont_train: (31258, 6)
  X_cat_val: (7815, 8), X_cont_val: (7815, 6)
  X_cat_test: (9769, 8), X_cont_test: (9769, 6)
Categorical feature cardinalities: [8, 16, 7, 14, 6, 5, 2, 41]

=== Tuning Hyperparameters for FT Transformer with Linear Embedding ===


[I 2025-03-04 15:58:10,796] Trial 0 finished with value: 0.31081729334208275 and parameters: {'d_token': 41, 'num_heads': 8, 'num_layers': 1, 'd_ffn': 106, 'lr': 0.0003410999348349744, 'dropout': 0.44689283934342133, 'n_bins': 36}. Best is trial 0 with value: 0.31081729334208275.
[I 2025-03-04 15:58:51,299] Trial 1 finished with value: 0.3239999077012462 and parameters: {'d_token': 69, 'num_heads': 5, 'num_layers': 2, 'd_ffn': 205, 'lr': 0.0031273098322414507, 'dropout': 0.029989994049630153, 'n_bins': 68}. Best is trial 0 with value: 0.31081729334208275.
[I 2025-03-04 15:59:40,620] Trial 2 finished with value: 0.31477202979787705 and parameters: {'d_token': 123, 'num_heads': 2, 'num_layers': 2, 'd_ffn': 166, 'lr': 0.0009762856449216884, 'dropout': 0.43636111200584454, 'n_bins': 96}. Best is trial 0 with value: 0.31081729334208275.
[I 2025-03-04 16:00:52,199] Trial 3 finished with value: 0.31045347596368483 and parameters: {'d_token': 78, 'num_heads': 2, 'num_layers': 1, 'd_ffn': 195, 

Best trial:
  Value (validation loss): 0.3027
  Params:
    d_token: 126
    num_heads: 4
    num_layers: 3
    d_ffn: 252
    lr: 0.00010993775924983645
    dropout: 0.13820584848558642
    n_bins: 41

=== Training FT Transformer with Linear Embedding (Tuned) ===
Epoch 1/100, Train Loss: 0.4107, Train Acc: 0.8065, Val Loss: 0.3324, Val Acc: 0.8429
Epoch 2/100, Train Loss: 0.3200, Train Acc: 0.8500, Val Loss: 0.3197, Val Acc: 0.8504
Epoch 3/100, Train Loss: 0.3133, Train Acc: 0.8527, Val Loss: 0.3161, Val Acc: 0.8517
Epoch 4/100, Train Loss: 0.3091, Train Acc: 0.8549, Val Loss: 0.3132, Val Acc: 0.8534
Epoch 5/100, Train Loss: 0.3085, Train Acc: 0.8558, Val Loss: 0.3181, Val Acc: 0.8536
Epoch 6/100, Train Loss: 0.3082, Train Acc: 0.8549, Val Loss: 0.3109, Val Acc: 0.8548
Epoch 7/100, Train Loss: 0.3064, Train Acc: 0.8564, Val Loss: 0.3149, Val Acc: 0.8516
Epoch 8/100, Train Loss: 0.3035, Train Acc: 0.8576, Val Loss: 0.3087, Val Acc: 0.8562
Epoch 9/100, Train Loss: 0.3029, Train Acc: 0.8

...

Best trial:
  Value (validation loss): 0.3064
  Params:
    d_token: 102
    num_heads: 2
    num_layers: 2
    d_ffn: 251
    lr: 0.00015200441693413884
    dropout: 0.32072996399966297
    n_bins: 91

=== Training sparse FT Transformer with Linear Embedding (Tuned) ===
Epoch 1/100, Train Loss: 0.4023, Train Acc: 0.8129, Val Loss: 0.3320, Val Acc: 0.8454
Epoch 2/100, Train Loss: 0.3276, Train Acc: 0.8476, Val Loss: 0.3253, Val Acc: 0.8470
Epoch 3/100, Train Loss: 0.3188, Train Acc: 0.8516, Val Loss: 0.3166, Val Acc: 0.8502
Epoch 4/100, Train Loss: 0.3137, Train Acc: 0.8526, Val Loss: 0.3155, Val Acc: 0.8520
Epoch 5/100, Train Loss: 0.3113, Train Acc: 0.8544, Val Loss: 0.3175, Val Acc: 0.8505
Epoch 6/100, Train Loss: 0.3109, Train Acc: 0.8541, Val Loss: 0.3132, Val Acc: 0.8522
Epoch 7/100, Train Loss: 0.3100, Train Acc: 0.8540, Val Loss: 0.3130, Val Acc: 0.8518
Epoch 8/100, Train Loss: 0.3088, Train Acc: 0.8559, Val Loss: 0.3135, Val Acc: 0.8520
Epoch 9/100, Train Loss: 0.3083, Train A

[I 2025-03-04 17:45:15,635] A new study created in memory with name: no-name-d0c09a39-f8e2-41e3-b14c-6f89cf16233d



=== Tuning Hyperparameters for sparse FT Transformer with Piecewise Linear Embedding ===


[I 2025-03-04 17:54:46,504] Trial 0 finished with value: 0.4499333832533129 and parameters: {'d_token': 58, 'num_heads': 4, 'num_layers': 2, 'd_ffn': 160, 'lr': 0.009929135345980139, 'dropout': 0.11062714321262168, 'n_bins': 71}. Best is trial 0 with value: 0.4499333832533129.
[I 2025-03-04 17:58:42,982] Trial 1 finished with value: 0.3146006738947284 and parameters: {'d_token': 58, 'num_heads': 7, 'num_layers': 1, 'd_ffn': 188, 'lr': 0.00041847257551783004, 'dropout': 0.2717393676873803, 'n_bins': 19}. Best is trial 1 with value: 0.3146006738947284.
[I 2025-03-04 18:03:00,761] Trial 2 finished with value: 0.37796270342603805 and parameters: {'d_token': 52, 'num_heads': 7, 'num_layers': 3, 'd_ffn': 206, 'lr': 0.0037844740333583646, 'dropout': 0.31220989812613237, 'n_bins': 62}. Best is trial 1 with value: 0.3146006738947284.
[I 2025-03-04 18:16:14,418] Trial 3 finished with value: 0.31273629468294883 and parameters: {'d_token': 69, 'num_heads': 5, 'num_layers': 1, 'd_ffn': 250, 'lr': 0

Best trial:
  Value (validation loss): 0.3127
  Params:
    d_token: 69
    num_heads: 5
    num_layers: 1
    d_ffn: 250
    lr: 0.0002958999543643145
    dropout: 0.05763815895475838
    n_bins: 91

=== Training sparse FT Transformer with Piecewise Linear Embedding (Tuned) ===
Epoch 1/100, Train Loss: 0.4285, Train Acc: 0.7863, Val Loss: 0.3522, Val Acc: 0.8343
Epoch 2/100, Train Loss: 0.3386, Train Acc: 0.8420, Val Loss: 0.3430, Val Acc: 0.8338
Epoch 3/100, Train Loss: 0.3314, Train Acc: 0.8473, Val Loss: 0.3367, Val Acc: 0.8462
Epoch 4/100, Train Loss: 0.3255, Train Acc: 0.8498, Val Loss: 0.3275, Val Acc: 0.8502
Epoch 5/100, Train Loss: 0.3219, Train Acc: 0.8507, Val Loss: 0.3269, Val Acc: 0.8507
Epoch 6/100, Train Loss: 0.3188, Train Acc: 0.8515, Val Loss: 0.3260, Val Acc: 0.8495
Epoch 7/100, Train Loss: 0.3180, Train Acc: 0.8518, Val Loss: 0.3235, Val Acc: 0.8518
Epoch 8/100, Train Loss: 0.3172, Train Acc: 0.8528, Val Loss: 0.3324, Val Acc: 0.8424
Epoch 9/100, Train Loss: 0.3152,